In [18]:
import pandas as pd
import numpy as np

def clean_excel_file(df):
    # Step 1: Rename Columns based on their expected order and content
    num_cols = df.shape[1]
    core_cols = ['SN', 'Polling_Station']

    # Check if the third column is likely 'Polling_Station_Name'
    # We expect this column to have 'name' in its header if it exists
    third_col_name = df.columns[2].lower()  # Lowercase the third column name to check properly
    if 'name' in third_col_name:
        core_cols.append('Polling_Station_Name')
        start_dynamic_cols = 4
    else:
        start_dynamic_cols = 3

    # Continue with the remaining expected column names
    core_cols += [f'col_{i}' for i in range(start_dynamic_cols, num_cols-5+1)] + \
                 ['Total_Valid_Votes', 'Total_Rejected_Votes', 'NOTA', 'Total', 'Total_Votes_Tendered']
    
    # Apply the new column names
    df.columns = core_cols

    if 'Polling_Station_Name' in df.columns:
        df.drop('Polling_Station_Name', axis=1, inplace=True)
        
    # Step 2: Clean all cells in the dataframe to remove unwanted characters
    df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))

    # Step 3: Replace 'nan' strings with actual NaN values
    df.replace(to_replace='^nan$', value=np.nan, regex=True, inplace=True)
    
    #sum
    

    # Step 4: Convert applicable columns to numeric types
    # Define columns to exclude from numeric conversion (text columns)
    exclude_columns = ['SN', 'Polling_Station']

    # Identify columns that should be processed (all except the excluded ones)
    columns_to_process = df.columns.difference(exclude_columns)

    # Apply numeric conversion only to the appropriate columns
    df[columns_to_process] = df[columns_to_process].apply(pd.to_numeric, errors='coerce')

    # Step 5: Filter out rows where any cell is non-numeric (where it should be numeric)
    df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
    start_col = 2  # 3rd column (index 2)
    end_col = len(core_cols) - 6   # 7th column (index 6)

    # Calculate the sum of values across specified columns
    sum_of_columns = df.iloc[:, start_col:end_col+1].sum(axis=1)
    # print(sum_of_columns)
    mismatch_mask = sum_of_columns == df['Total_Valid_Votes']
    # mismatch_count = mismatch_mask.sum()
    mismatch_count = (~mismatch_mask).sum()
    print(mismatch_count)
    return mismatch_count

In [41]:
import os

def check_all_files(folder_path,year):
    mismatch_counts = {}
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.xlsx') or filename.endswith('.xls'):
            try:
                df = pd.read_excel(os.path.join(folder_path, filename))
                # file_path = os.path.join(folder_path, filename)
                mismatches = clean_excel_file(df)
                if mismatches > 0:  # Store only if mismatches are greater than 0
                    parsed_filename = filename.split('.')[0].split('_')[-1]
                    mismatch_counts[parsed_filename] = mismatches
                    results.append({'Year': year, 'Filename': f"{parsed_filename}.xlsx", 'Mismatches': mismatches})
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                parsed_filename = filename.split('.')[0].split('_')[-1]
                
                results.append({'Year': year,'Filename': f"{parsed_filename}.xlsx", 'Mismatches': "Error"})


    results_df = pd.DataFrame(results)
    mismatched_count = "results/mismatched_count"
    results_df.to_csv(os.path.join(mismatched_count, 'mismatch_results_maharastra_GE_2014.csv'), index=False)
    print("Results saved to mismatch_results.csv")

    return mismatch_counts


In [43]:
folder_path = 'results/Parsed_Excel/Maharastra/LokSabha Election 2014'
mismatch_results = check_all_files(folder_path,2014)
# print(mismatch_results)

/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace='^nan$', value=np.nan, regex=True, inplace=True)
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x

16
20


/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]


1


/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]


...

/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df =

0
23


/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]


269
7
1
Results saved to mismatch_results.csv


/tmp/ipykernel_213204/3639310771.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_213204/3639310771.py:48: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[columns_to_process].applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
